# Experiment settings

Dataset: KOSPI index from Jan, 2011 to Dec, 2020\
Included data in default: Daily log return, Daily trading range, Log volume change\

# Data analysis

- Augment Dickey Fuller test -> check for presence of unit root of the daily log return.
ADF test statistics value: -18.6316
p-value: 2.0573e-30

Therefore, the daily log return is stationary.

## Model selection part

- EWMA model
    * Tuned parameter: Weight decay (lambda)
    * Range of changing the param: 0.9 ~ 0.99
    * Applied to train set which is 80% of total data

Simulation results

| Weight Decay  | RMSE          | R2 score  | MAPE      |
| :------------ |:-------------:| :--------:| ---------:|
| 0.90          | 0.0363        | 0.913     | 0.103     |
| 0.91          | 0.0340        | 0.923     | 0.098      |
| 0.92          | 0.0330        | 0.928     | 0.094     |
| 0.93          | 0.0338        | 0.924     | 0.092     |
| 0.94          | 0.0370        | 0.910     | 0.094     |
| 0.95          | 0.0425        | 0.880     | 0.104     |
| 0.96          | 0.0505        | 0.831     | 0.121     |
| 0.97          | 0.0610        | 0.753     | 0.149     |
| 0.98          | 0.0748        | 0.630     | 0.195     |
| 0.99          | 0.0926        | 0.432     | 0.253     |

The best fit model (Decay: 0.93)
![title](images/EWMA.png)

- GARCH model selection

GARCH model with different lags

For selecting appropriate lags for GARCH models, partial autocorrelation function is used

![title](images/kospi_pacf.png)

Candidates: GARCH(1,1), GARCH(2,2), GARCH(3,3), EGARCH(1,1), EGARCH(2,2), EGARCH(3,3)
The candidates are nominated by refering pacf plot. The pacf plot indicates that from lag 1 to lag 3
is out of significance level so that the past 3day values are relatively related to the current
value.

Strategy: The models are fitted by train set. Find the best model among the candidates first using AIC, BIC and Log-Likelihood.
Then, tune the p and q parameters of the chosen one referring to p-value described at the model summary.

AIC, BIC formula: [-2logL + kp] where L is the likelihood function, p is the number of params
k is 2 for AIC, and log(n) for BIC.

Model selection criteria

| Models        | AIC        | BIC       | Log-Likelihood |
| :------------ |:----------:| :--------:| --------------:|
| GARCH(1,1)    | 4789.65    | 4823.13   | -2388.83       |
| GARCH(2,2)    | 4789.29    | 4833.93   | -2386.64       |
| GARCH(3,3)    | 4786.71    | 4842.52   | -2383.36       |
| EGARCH(1,1)   | 4792.16    | 4825.65   | -2390.08       |
| EGARCH(2,2)   | 4790.80    | 4835.45   | -2387.40       |
| EGARCH(3,3)   | 4787.78    | 4843.59   | -2383.89       |

There are no significant difference at AIC, BIC and Log-Likelihood.
So I did all permutations of lags for forecasting.

Results of GARCHs and EGARCHs

All other different permutations of GARCH lags shows the less effectiveness than GARCH(1,1) model
The GARCH(1,1) model has the least errors. Also, EGARCH(1,1) ahs the least errors among them.

| Models        | RMSE       | MAPE      |
| :------------ | :--------: | --------:  |
| GARCH(1,1)    | 0.039      | 0.107     |
| GARCH(2,2)    | 0.041      | 0.113     |
| GARCH(3,3)    | 0.041      | 0.115     |
| GJR-GARCH(1,1)   | 0.0517     | 0.171   |
| GJR-GARCH(2,2)   | 0.0515     | 0.168   |
| GJR-GARCH(3,3)   | 0.0507     | 0.164   |
| EGARCH(1,1)   | 0.0679     | 0.192   |
| EGARCH(2,2)   | 0.0668     | 0.190   |
| EGARCH(3,3)   | 0.0670     | 0.191   |


![title](images/garch11.png)

![title](images/egarch11.png)

Result of LSTM model

Single layer with 10 nodes.

- RMSE: 0.0306
- MAPE: 0.0892
![title](images/single_layer_model_pred.png)

The results are 1 day ahead forecast with 22 rolling window. I am going to do 1 day forecast
with 15 rolling window, and 5 day ahead forecast until this week.
Then will move on write the report.